In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [11]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, docs_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [12]:
wiki.name

'wikipedia'

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings

loader = WebBaseLoader("https://docs.smith.langchain.com")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)
embeddings = OllamaEmbeddings(model="mxbai-embed-large")
vectordb = FAISS.from_documents(documents, embeddings)
retriever = vectordb.as_retriever()
retriever



VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F5D5349160>, search_kwargs={})

In [8]:
#create tool to retrieval of all documents
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith_search","search for information about langsmith. for any question about langsmith you must use this tool")



In [9]:
retriever_tool

Tool(name='langsmith_search', description='search for information about langsmith. for any question about langsmith you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001F5FDC847C0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F5D5349160>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001F5FDC84900>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F5D5349160>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, parti

In [10]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, docs_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [13]:
tools=[wiki,arxiv,retriever_tool]

In [14]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'e:\\LANGCHAIN REBUILT\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='langsmith_search', description='search for information about langsmith. for any question about langsmith you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001F5FDC847C0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectors

In [ ]:
"""
from dotnet import load_dotenv
load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenaAI

llm = ChatOpenaAI(model = "gpt-3.5-turbo-0125", temperature =0)
"""

In [ ]:
"""
from langchain import hub
#get the prompt to use - can be modifiedd
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages
"""

In [ ]:
### agents are used to use a language model to chooose a sequence of actions to take. Language model is used as a reasoning agent to determine which actions to take and in which order
#from langchain.agents import create_openai_tools_agent
#agent = create_openai_tools_agent(llm, tools, prompt)

In [ ]:
## agent executer
"""
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools,verbose=True)
agent_executor
agent_executor.invoke({"input":"Tell me about Langsmith"})
"""

In [ ]:
"""
agent_executor.invoke({"input":"Tell me about MAchineLearning"})
"""

In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain.agents import Tool
from langchain_community.llms import Ollama

llm = Ollama(model="mistral", temperature=0)

tools = [
    Tool(name="arxiv", func=arxiv.run, description="Research papers"),
    Tool(name="wiki", func=wiki.run, description="Wikipedia queries"),
    Tool(name="retriever", func=retriever_tool.run, description="LangSmith queries"),
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  #weaker than OpenAI but Mistral-friendly because I dont have PAISA
    verbose=True
)

response = agent.run("Tell me about LangSmith")
print("Final Answer:", response)

from langchain.prompts import PromptTemplate

agent_prompt = PromptTemplate.from_template("""
You are an intelligent assistant with access to tools.

TOOLS:
- arxiv: use for research papers.
- wiki: use for Wikipedia queries.
- retriever: use for LangSmith related queries.

When deciding, follow this format strictly:
Thought: reason about what to do
Action: tool name (one of [arxiv, wiki, retriever])
Action Input: the input to the tool
Observation: the tool's result
Final Answer: the final response to the user
""")





> Entering new AgentExecutor chain...
 To find information about LangSmith, I should use the retriever tool. The input for the retriever tool can be a specific query related to LangSmith.

Action: retriever
Action Input: {'query': 'LangSmith'}
Observation: Get started with LangSmith - Docs by LangChainDocs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationOverviewQuickstartsTrace an applicationEvaluate an applicationTest promptsAPI & SDKsAPI referencePython SDKJS/TS SDKPricingPlansPricing FAQDocs by LangChain home pagePythonSearch...⌘KAsk AIGitHubForumForumSearch...NavigationGet started with LangSmithGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGitHubForumGet started with LangSmithCopy pageCopy pageLangSmith is a platform for building production-grade LLM applications. Monitor and evaluate your application, so you can ship quickly a

In [24]:
response = agent.run("Tell me about Machine Learning")
print("Final Answer:", response)



> Entering new AgentExecutor chain...
 To answer this question, I will use a combination of resources. First, I'll look up an overview of machine learning on Wikipedia to get a general understanding. Then, I'll search for research papers related to machine learning to delve deeper into the subject.

Action: wiki
Action Input: Machine Learning
Observation: Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.
ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, an

In [ ]:
"""ZERO_SHOT_REACT_DESCRIPTION

It’s one of the built-in agent types in LangChain.

Zero-shot → means the model is not trained on your tools beforehand; it figures out which tool to call just from the tool descriptions you give.

ReAct → stands for Reasoning + Acting. The model doesn’t just directly answer, it follows a loop:

Reason about the question (decides what tool is needed).

Act by calling a tool (e.g., search, math, retriever).

Looks at the tool output.

Decides next step until it has a final answer.

Description → it uses the tool descriptions you provide to decide when and how to use them."""